In [1]:
def dominates(p1, p2):
    return (p1['x'] <= p2['x'] and p1['y'] >= p2['y']) and (p1['x'] < p2['x'] or p1['y'] > p2['y'])

def sequential_scan_skyline(dataset):
    skyline = []
    start_time = time.time()
    for point in dataset:
        dominated = False
        for s_point in skyline:
            if dominates(s_point, point):
                dominated = True
                break
        if not dominated:
            skyline = [s_point for s_point in skyline if not dominates(point, s_point)]
            skyline.append(point)
    total_time = time.time() - start_time
    return skyline, total_time
if __name__ == '__main__':
    dataset_file = 'city1.txt'
    output_file = 'sequential_scan_skyline_output.txt'

    dataset = read_dataset(dataset_file)

    skyline, total_time = sequential_scan_skyline(dataset)

    with open(output_file, 'w') as file:
        for point in skyline:
            file.write(f"id={point['id']}, x={point['x']}, y={point['y']}\n")
        file.write(f"Total Time: {total_time:.4f} seconds\n")


NameError: name 'read_dataset' is not defined

In [ ]:
def bbs_algorithm_skyline(dataset):
    def mbr_dominance(mbr1, mbr2):
        return (mbr1[0] <= mbr2[0] and mbr1[1] >= mbr2[1]) and (mbr1[0] < mbr2[0] or mbr1[1] > mbr2[1])

    rtree = RTree()
    for point in dataset:
        rtree.insert(point)

    pq = []
    skyline = []
    heapq.heappush(pq, (0, rtree.root))

    start_time = time.time()
    while pq:
        _, node = heapq.heappop(pq)
        if node.is_leaf:
            for point in node.points:
                if not any(dominates(s_point, point) for s_point in skyline):
                    skyline = [s_point for s_point in skyline if not dominates(point, s_point)]
                    skyline.append(point)
        else:
            for child in node.children:
                if not any(mbr_dominance(s_point['mbr'], child.mbr) for s_point in skyline):
                    heapq.heappush(pq, (0, child))
    total_time = time.time() - start_time
    return skyline, total_time

if __name__ == '__main__':
    dataset_file = 'datasets/city1.txt'
    output_file = 'output/bbs_algorithm_skyline_output.txt'

    dataset = read_dataset(dataset_file)

    skyline, total_time = bbs_algorithm_skyline(dataset)

    with open(output_file, 'w') as file:
        for point in skyline:
            file.write(f"id={point['id']}, x={point['x']}, y={point['y']}\n")
        file.write(f"Total Time: {total_time:.4f} seconds\n")


In [ ]:
def dc_bbs_algorithm_skyline(dataset):
    left_dataset, right_dataset = divide_dataset(dataset, axis=0)

    left_rtree = RTree()
    for point in left_dataset:
        left_rtree.insert(point)

    right_rtree = RTree()
    for point in right_dataset:
        right_rtree.insert(point)

    def dc_skyline(rtree):
        return bbs_algorithm_skyline([rtree])

    left_skyline, _ = dc_skyline(left_rtree)
    right_skyline, _ = dc_skyline(right_rtree)

    combined_skyline = left_skyline + right_skyline
    final_skyline, total_time = sequential_scan_skyline(combined_skyline)

    return final_skyline, total_time

if __name__ == '__main__':
    dataset_file = 'datasets/city1.txt'
    output_file = 'output/dc_bbs_algorithm_skyline_output.txt'

    dataset = read_dataset(dataset_file)

    skyline, total_time = dc_bbs_algorithm_skyline(dataset)

    with open(output_file, 'w') as file:
        for point in skyline:
            file.write(f"id={point['id']}, x={point['x']}, y={point['y']}\n")
        file.write(f"Total Time: {total_time:.4f} seconds\n")
